**Numerical Methods - Homework 3**
==
**Autor:** Weatherly Moura Albuquerque
==
**Matrícula:** 511792
==

In [ ]:
#@title Código para importações (Executar primeiro) {display-mode: "form"}

# This code will be hidden when the notebook is loaded.
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import time
from datetime import datetime

##Definição de funções a ser usadas no decorrer da execução das questões.

In [ ]:
#@title Spline Quadrática {display-mode: "form"}

# This code will be hidden when the notebook is loaded.

def splinequadratica(vetx, vety):
  y = np.copy(vety)
  x =np.copy(vetx)
  n = len(x)-1
  incog = 3*n-1
  a, b, c = [], [], []                    
  vet = np.zeros(incog, dtype = 'float')  
  matA = np.zeros((incog, incog), dtype = 'float') 
  col = 0
  idx_vet = 1
  lin = 2
  for i in range(0, n-1):
    if i == 0:
      matA[0, col], matA[0, col+1], vet[0] = x[idx_vet], 1, y[idx_vet]
      col += 2
      matA[1, col], matA[1, col+1], matA[1, col+2], vet[1] = x[idx_vet]**2, x[idx_vet], 1, y[idx_vet]
    else:
      idx_vet += 1
      matA[lin, col], matA[lin, col+1], matA[lin, col+2], vet[lin] = x[idx_vet]**2, x[idx_vet], 1, y[idx_vet]
      col += 3
      matA[lin+1, col], matA[lin+1, col+1], matA[lin+1, col+2], vet[lin+1] = x[idx_vet]**2, x[idx_vet], 1, y[idx_vet]
      lin += 2
  matA[lin, 0], matA[lin, 1], vet[lin] = x[0], 1, y[0]
  matA[lin+1, incog-3], matA[lin+1, incog-2], matA[lin+1, incog-1], vet[lin+1] = x[n]**2, x[n], 1, y[n]
  lin += 2
  idx_vet = 1
  coli = 0
  for i in range(0, n-1):
    if i == 0:
      matA[lin, coli], matA[lin, coli+2], matA[lin, coli+3]  = 1, -2*x[idx_vet], -1
      lin += 1
      idx_vet += 1
      coli += 2
    else:
      matA[lin, coli], matA[lin, coli+1], matA[lin, coli+3], matA[lin, coli+4] = 2*x[idx_vet], 1, -2*x[idx_vet], -1
      lin += 1
      idx_vet += 1
      coli += 3
  a.append(0)
  resp = np.linalg.solve(matA, vet)
  cont = 0
  while(cont <= len(resp)-3):
    if cont == 0:
      b.append(resp[cont])
      c.append(resp[cont+1])
      cont += 2
    else:
      a.append(resp[cont])
      b.append(resp[cont+1])
      c.append(resp[cont+2])
      cont += 3
  return a, b, c

In [ ]:
#@title Spline Cúbica {display-mode: "form"}

# This code will be hidden when the notebook is loaded.

def splinecubica(vetx, vety):
  x = np.copy(vetx)
  y = np.copy(vety)
  n = len(x)
  incog = 4*(n-1)
  a, b, c, d = [], [], [], []
  vet = np.zeros(incog, dtype = 'float')
  matA = np.zeros((incog, incog), dtype = 'float') 
  col = 0
  lin = 0
  idx_vet = 1
  for i in range(0, n-2):
    matA[lin, col], matA[lin, col+1], matA[lin, col+2], matA[lin, col+3], vet[lin] = (x[idx_vet])**3, x[idx_vet]**2, x[idx_vet], 1, y[idx_vet]
    col += 4
    matA[lin+1, col], matA[lin+1, col+1], matA[lin+1, col+2], matA[lin+1, col+3], vet[lin+1] = x[idx_vet]**3, x[idx_vet]**2, x[idx_vet], 1, y[idx_vet]
    idx_vet += 1
    lin += 2
  matA[lin, 0], matA[lin, 1], matA[lin, 2], matA[lin, 3] , vet[lin] = x[0]**3, x[0]**2, x[0], 1, y[0]
  matA[lin+1, incog-4], matA[lin+1, incog-3], matA[lin+1, incog-2],  matA[lin+1, incog-1], vet[lin+1] = x[n-1]**3, x[n-1]**2, x[n-1], 1, y[n-1]
  lin += 2
  idx_vet = 1
  coli = 0
  for i in range(0, n-2):
    matA[lin, coli], matA[lin, coli+1], matA[lin, coli+2], matA[lin, coli+4], matA[lin, coli+5], matA[lin, coli+6] = 3*(x[idx_vet]**2), 2*x[idx_vet], 1, -3*(x[idx_vet]**2), -2*x[idx_vet], -1
    lin += 1
    idx_vet += 1
    coli += 4
  idx_vet = 1
  coli = 0
  for i in range(0, n-2):
    matA[lin, coli], matA[lin, coli+1], matA[lin, coli+4], matA[lin, coli+5] = 6*(x[idx_vet]), 2, -6*(x[idx_vet]), -2
    lin += 1
    idx_vet += 1
    coli += 4
  matA[lin, 0], matA[lin, 1] = 6*x[0], 2
  matA[lin+1, incog-4], matA[lin+1, incog-3] = 6*x[n-1], 2
  resp = np.linalg.solve(matA, vet)
  cont = 0
  while (cont<=len(resp)-4):
    a.append(resp[cont])
    b.append(resp[cont+1])
    c.append(resp[cont+2])
    d.append(resp[cont+3])
    cont += 4
  return a, b, c, d

In [ ]:
#@title Upsampling {display-mode: "form"}

# This code will be hidden when the notebook is loaded.

def acha_polinomios(coeficientes):
  polinomios = []
  for i in range(0, len(coeficientes), 4):
    polinomios.append(coeficientes[i:i+4])
  return polinomios

def acha_as_vendas_diarias(polinomios):
  valores = []
  for i in range(0,len(polinomios)):
    polinomio = polinomios[i]
    for j in range(1,31):
      valores.append(polinomio[0][0]*j**2 + polinomio[1][0]*j + polinomio[2][0])
  return valores

In [ ]:
#@title Avaliador de spline {display-mode: "form"}

# This code will be hidden when the notebook is loaded.

def verifica_spline(ponto, vetor, X, Y, Z, W=0):
  valor = 'fora do intervalo'
  if W!=0: #spline cubica
    for i in range(1, len(vetor)):
      if (ponto>= vetor[i-1]) and (ponto <= vetor[i]):
        valor = X[i-1]*(ponto**3) + Y[i-1]*(ponto**2) + Z[i-1]*(ponto) + W[i-1]
  else: #spline quadratica
    for i in range(1, len(vetor)):
      if (ponto>= vetor[i-1]) and (ponto <= vetor[i]):
        valor = X[i-1]*(ponto**2) + Y[i-1]*ponto + Z[i-1]

  return valor

## Questão 1 
Considere os dados abaixo.
===
Dados|||||||
--|--|--|--|--|--|-- 
X|1.6|2|2.5|3.2|4|4.5
Y|2|8|14|15|8|2

Uma spline quadrática é uma curva suave que é construída usando trechos de polinômios quadráticos que se encaixam perfeitamente em pontos dados. É chamada de spline quadrática porque cada trecho da curva é um polinômio quadrático. As splines quadráticas são úteis em aplicações em que é necessário aproximar uma curva por meio de pontos dados ou interpolar uma curva a partir de dados experimentais. Elas também podem ser usadas para suavizar gráficos e reduzir o ruído em dados.

In [ ]:
#@title Letra A - Obter e plotar o gráfico da spline quadrática que passa pelos pontos dados; {display-mode: "form"}

# This code will be hidden when the notebook is loaded.
xi = [1.6,2,2.5,3.2,4,4.5]
yi = [2,8,14,15,8,2]


inicio = time.time()
a,b,c = splinequadratica(xi, yi)
fim = time.time()

print(f"O tempo de duração do calculo da spline quadrática foi de {fim - inicio} segundos"" \n")


yvalue =[]
xvalue = np.linspace(1.6,4.5,1000) 
for k in range(0, len(xvalue)):
  yvalue.append(verifica_spline(xvalue[k], xi, a, b, c))

plt.plot(xvalue,yvalue, 'r')
plt.plot(xi,yi,'o')
plt.grid()
plt.show()

A spline cúbica é uma curva suave que é usada na interpolação de pontos. É uma ferramenta comumente utilizada em aplicações de engenharia e ciência da computação para aproximar e suavizar conjuntos de dados.

A spline cúbica é formada por pequenas curvas de terceiro grau, que são colocadas juntas de forma suave e contínua. Isso permite que a curva seja facilmente modificada para ajustar-se aos dados, sem que haja pontos de inflexão ou mudanças bruscas de direção.

Essa ferramenta é muito útil em aplicações que requerem precisão, como na modelagem de superfícies em engenharia mecânica ou na criação de gráficos de tendência em dados financeiros. Também é utilizada em técnicas de interpolacão de imagens, como a suavização de bordas em fotografias.

Ao contrário de outros métodos de interpolação, como as splines lineares ou quadráticas, a spline cúbica permite maior flexibilidade e precisão na aproximação dos dados. No entanto, ela também é mais complexa de se calcular e exige mais recursos computacionais.

In [ ]:
#@title Letra B -  Obter e plotar o gráfico da spline cúbica que passa pelos pontos dados; {display-mode: "form"}

# This code will be hidden when the notebook is loaded.

inicio = time.time()
a,b,c,d = splinecubica(xi, yi)
fim = time.time()

print(f"O tempo de duração do calculo da spline cúbica foi de {fim - inicio} segundos"" \n")


yvalue =[]
xvalue = np.linspace(1.6,4.5,1000) 
for k in range(0, len(xvalue)):
  yvalue.append(verifica_spline(xvalue[k], xi, a, b, c,d))

plt.plot(xvalue,yvalue, 'r')
plt.plot(xi,yi,'o')
plt.grid()
plt.show()

In [ ]:
#@title Letra C - Estimar o valor de f(2,8) pelos dois métodos referidos acima {display-mode: "form"}

# This code will be hidden when the notebook is loaded.

xi = [1.6,2,2.5,3.2,4,4.5]
yi = [2,8,14,15,8,2]
xf = 2.8
a,b,c = splinequadratica(xi,yi)
print('Para a spline quadrática, temos:', verifica_spline(xf, xi, a, b, c))


xi = [1.6,2,2.5,3.2,4,4.5]
yi = [2,8,14,15,8,2]
xf = 2.8
a,b,c,d = splinecubica(xi,yi)
print('Para a spline cúbica, temos:', verifica_spline(xf, xi, a, b, c, d))

## Questão 02

Use interpolação por spline cúbica para a representação dos dados abaixo, e determine o valor de 𝑥 que corresponde a 𝑓(𝑥) = 0,23 para os seguintes dados tabulados através da solução de um problema de raiz do correspondente polinômio cúbico.
===
Dados|||||||
--|--|--|--|--|--|-- 
X|2|3|4|5|6|7
Y|0.5|0.3333|0.25|0.2|0.1667|0.1429

In [ ]:
#@title Resolução {display-mode: "form"}

# This code will be hidden when the notebook is loaded.

xi = [0.5,0.3333,0.25,0.2,0.1667,0.1429]
xi = np.sort(xi)
yi = [7,6,5,4,3,2]

inicio = time.time()
a,b,c,d = splinecubica(xi, yi)
fim = time.time()

print(f"O tempo de duração do calculo da spline cúbica foi de {fim - inicio} segundos"" \n")

xf = 0.23
avaliapol(xf, xi, a, b, c,d)

def func(x):
  return a[3]*x**3 + b[3]*x**2 +c[3]*x + d[3]-0.23

func(0.23)

print(f"A determinação do valor correspondente a f(x) = 0,23 é: {func(0.23)}")

## Questão 03 
Efetue o download do dataset Shampoo Sales Dataset1, no ambiente Python, o qual descreve o número de vendas mensais de shampoo ao longo de um período de três anos, e utilizando o processo de interpolação por spline cúbica, efetue o processo de upsampling para obter estimativas diárias de vendas. 

Upsampling é um processo em que se aumenta a taxa de amostragem de uma série temporal, aumentando assim o número de amostras da série. Isso pode ser feito inserindo valores interpolados entre as amostras existentes ou usando algum tipo de filtro para suavizar a série temporal. O objetivo do upsampling é aumentar a resolução temporal da série, permitindo uma análise mais detalhada dos dados. No entanto, é importante lembrar que o upsampling não adiciona informação nova à série, apenas aumenta o número de amostras existentes.

In [ ]:
#@title Resolução {display-mode: "form"}

# This code will be hidden when the notebook is loaded.

url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/shampoo.csv'
data_frame = pd.read_csv(url)

def parser_to_datetime(x):
 return datetime.strptime('200'+x, '%Y-%m')
 
series = pd.read_csv(url, header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser_to_datetime)
upsampled = series.resample('D')

meses = np.linspace(0,35,35) 
[a3, b3, c3, d3] = splinecubica(meses, data_frame.Sales)

intervalo = np.linspace(0,35,365*3)
valores =[]
for i in range(0, len(intervalo)):
  valores.append(verifica_spline(intervalo[i], meses, a3, b3, c3, d3))

dia = 1
ano = 1
for valor in valores:
  if (dia % 366 == 0):
    dia = 1
    ano += 1
  print(f'O valor diário do dia {dia} e do ano {ano} é: {valor:.0f}')
  dia += 1